# Create meeting minutes from an Audio file

For this project, I have selected a portion of a Denver City Council meeting to transcribe and summarize. You can download the audio file here: 
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing

If you'd like to work with the original dataset, you can find it on Hugging Face:[here](https://huggingface.co/datasets/huuuyeah/meetingbank) and the audio can be downloaded [here](https://huggingface.co/datasets/huuuyeah/MeetingBank_Audio/tree/main).

## Project Objective
The goal of this project is to convert an audio recording into structured meeting minutes, including key discussion points, action items, and summaries.

You can use the Denver meeting audio or record your own meeting to test the system!

## Note
 I am working on google colab due to the GPU available, you can also start your own google collab session and start working on this.





In [ ]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2 openai-whisper

In [ ]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import whisper

In [3]:
# Constants

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
model = whisper.load_model("base")

In [ ]:
# New capability - connect this Colab to your Google Drive. Add the file in the MyDrive folder in your personal drive


drive.mount("/content/drive")
audio_filename = "/content/drive/MyDrive/denver_extract.mp3"

In [6]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [7]:
# Sign in to OpenAI using Secrets in Colab
MODEL = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")


In [ ]:
# Use the Whisper OpenAI model to convert the Audio to Text

audio_file = open(audio_filename, "rb")
transcription =  model.transcribe (audio_filename)
print(transcription)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [25]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [26]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

In [28]:
response = tokenizer.decode(outputs[0])

In [ ]:
display(Markdown(response))